In [ ]:
"""
LINCS数据加载和预处理脚本 - DrugReflector论文复现版（放宽筛选条件）
基于Science 2025补充材料，但适当放宽了一些筛选条件以避免过度过滤
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import cosine, pdist, squareform
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')


class LINCSDataLoader:
    """加载和预处理LINCS L1000数据 - 优化版（放宽筛选条件）"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.signatures = None
        self.decompressed_files = []
        
    def load_gene_info(self, dataset='GSE92742'):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / f"{dataset}_Broad_LINCS_gene_info.txt.gz"
        
        gene_info = pd.read_csv(gene_file, sep='\t', compression='gzip')
        
        # 筛选landmark genes (pr_is_lm == 1)
        landmark_genes = gene_info[gene_info['pr_is_lm'] == 1].copy()
        
        print(f"Total genes: {len(gene_info)}")
        print(f"Landmark genes: {len(landmark_genes)}")
        
        self.gene_info = landmark_genes
        return landmark_genes
    
    def load_cell_info(self, dataset='GSE92742'):
        """加载细胞系信息"""
        cell_file = self.data_dir / f"{dataset}_Broad_LINCS_cell_info.txt.gz"
        
        cell_info = pd.read_csv(cell_file, sep='\t', compression='gzip')
        
        print(f"Total cell lines: {len(cell_info)}")
        print(f"Unique cell IDs: {cell_info['cell_id'].nunique()}")
        
        self.cell_info = cell_info
        return cell_info
    
    def decompress_gzip_file(self, gzip_file):
        """解压gzip文件到_decompressed文件夹"""
        gzip_file = str(gzip_file)
        
        decompressed_dir = Path(self.data_dir) / "_decompressed"
        decompressed_dir.mkdir(exist_ok=True)
        
        original_name = Path(gzip_file).stem
        output_path = decompressed_dir / original_name
        
        if output_path.exists():
            print(f"✓ Found existing decompressed file: {output_path.name}")
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        print(f"💾 Decompressing to: {decompressed_dir}")
        
        try:
            print(f"📦 Decompressing {Path(gzip_file).name}...")
            source_size = Path(gzip_file).stat().st_size / (1024**3)
            print(f"   Source size: ~{source_size:.1f} GB")
            
            with gzip.open(gzip_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            output_size = output_path.stat().st_size / (1024**3)
            print(f"✓ Decompressed successfully!")
            print(f"✓ Output size: ~{output_size:.1f} GB")
            
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        except Exception as e:
            print(f"\n❌ Decompression failed: {e}")
            if output_path.exists():
                try:
                    output_path.unlink()
                except:
                    pass
            raise RuntimeError(f"Failed to decompress {gzip_file}: {e}")
    
    def read_gctx(self, gctx_file):
        """读取GCTX文件 (HDF5格式)"""
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        if gctx_file.endswith('.gz'):
            print("⚠️  Detected gzip compressed file")
            gctx_file = self.decompress_gzip_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            print(f"📊 Loading matrix from HDF5...")
            matrix = f['/0/DATA/0/matrix'][:]
            print(f"✓ Matrix shape: {matrix.shape}")
            
            print(f"📋 Loading metadata...")
            
            # 读取基因元数据（来自 ROW）
            gene_meta = {}
            for key in f['/0/META/ROW'].keys():
                data = f[f'/0/META/ROW/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    gene_meta[key] = data.astype(str)
            
            # 读取样本元数据（来自 COL）
            sample_meta = {}
            for key in f['/0/META/COL'].keys():
                data = f[f'/0/META/COL/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    sample_meta[key] = data.astype(str)
        
        sample_df = pd.DataFrame(sample_meta)
        gene_df = pd.DataFrame(gene_meta)
        
        print(f"\n✓ Data loaded successfully:")
        print(f"  Matrix: {matrix.shape} (samples × genes)")
        print(f"  Samples: {len(sample_df)}")
        print(f"  Genes: {len(gene_df)}")
        
        assert matrix.shape[0] == len(sample_df), \
            f"Matrix rows ({matrix.shape[0]}) != sample metadata ({len(sample_df)})"
        assert matrix.shape[1] == len(gene_df), \
            f"Matrix cols ({matrix.shape[1]}) != gene metadata ({len(gene_df)})"
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self, dataset='GSE92742'):
        """加载Level 4数据 (Z-score normalized)"""
        level4_pattern = self.data_dir / f"{dataset}_Broad_LINCS_Level4_ZSPCINF_mlr12k_n*x12328.gctx.gz"
        
        print(f"🔍 Searching for Level 4 file...")
        print(f"   Pattern: {level4_pattern.name}")
        files = glob.glob(str(level4_pattern))
        
        if not files:
            raise FileNotFoundError(
                f"❌ No Level 4 file found matching {level4_pattern}"
            )
        
        level4_file = files[0]
        print(f"✓ Found: {Path(level4_file).name}")
        
        # 读取GCTX
        matrix, sample_meta, gene_meta = self.read_gctx(level4_file)
        
        # 只保留landmark genes
        print(f"\n🔬 Filtering to landmark genes...")
        if self.gene_info is None:
            self.load_gene_info(dataset)
        
        landmark_ids = set(self.gene_info['pr_gene_id'].astype(str).values)
        print(f"   Landmark genes to find: {len(landmark_ids)}")
        print(f"   Total genes in data: {len(gene_meta)}")
        
        gene_mask = gene_meta['id'].isin(landmark_ids)
        n_matched = gene_mask.sum()
        print(f"   ✓ Matched: {n_matched} genes")
        
        if n_matched == 0:
            raise ValueError("No landmark genes matched!")
        
        print(f"\n🎯 Applying filter...")
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"   ✓ Final matrix shape: {matrix.shape}")
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        return matrix, sample_meta, gene_meta
    
    def calculate_cosine_similarity_to_nearest_replicate(
        self, 
        matrix: np.ndarray, 
        pert_ids: pd.Series
    ) -> np.ndarray:
        """
        计算每个样本与其最近replicate的余弦相似度（向量化优化）
        """
        print(f"\n📊 Calculating cosine similarity to nearest replicate...")
        n_samples = len(pert_ids)
        nearest_similarities = np.zeros(n_samples)
        
        unique_perts = pert_ids.unique()
        print(f"   Processing {len(unique_perts)} compounds...")
        
        for i, pert_id in enumerate(unique_perts):
            if (i + 1) % 1000 == 0:
                print(f"   Progress: {i + 1}/{len(unique_perts)}")
            
            pert_mask = pert_ids == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            if len(pert_indices) < 2:
                # 只有一个样本，设为0（会被过滤）
                nearest_similarities[pert_indices] = 0.0
                continue
            
            pert_data = matrix[pert_mask]
            
            # 使用sklearn快速计算成对余弦相似度
            sim_matrix = cosine_similarity(pert_data)
            
            # 将对角线设为-inf，避免样本与自己比较
            np.fill_diagonal(sim_matrix, -np.inf)
            
            # 对每个样本找到最大相似度（最近的replicate）
            max_sims = np.max(sim_matrix, axis=1)
            
            nearest_similarities[pert_indices] = max_sims
        
        print(f"   ✓ Calculated similarities for {n_samples} samples")
        print(f"   Mean similarity: {nearest_similarities.mean():.4f}")
        print(f"   Median similarity: {np.median(nearest_similarities):.4f}")
        
        return nearest_similarities
    
    def prepare_training_data(
        self, 
        min_observations_per_compound=3,      # 放宽：5 → 3
        min_replicate_similarity=0.10,        # 放宽：0.12 → 0.10
        dose_range=(0.3, 30.0),               # 放宽：(1.0, 20.0) → (0.3, 30.0)
        valid_timepoints=['6h', '24h', '6 h', '24 h'],  # 增加格式
        min_cell_lines=3,                     # 放宽：5 → 3
        max_cell_lines=50,                    # 放宽：40 → 50
        remove_dos=True,
        skip_missing_metadata=True            # 新增：缺失元数据时跳过而不是失败
    ):
        """
        准备训练数据，适当放宽论文的过滤流程
        
        主要调整：
        1. min_observations_per_compound: 5 → 3（放宽）
        2. min_replicate_similarity: 0.12 → 0.10（放宽）
        3. dose_range: 1-20µM → 0.3-30µM（放宽）
        4. min_cell_lines: 5 → 3（放宽）
        5. max_cell_lines: 40 → 50（放宽）
        6. 增加时间点格式的容错
        7. 缺失元数据时跳过过滤而不是报错
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta'].copy()
        col_meta = self.signatures['col_meta']
        
        print("\n" + "=" * 80)
        print("🔍 DRUGREFLECTOR QUALITY CONTROL PIPELINE (RELAXED)")
        print("   Based on Science 2025 SI with relaxed filtering criteria")
        print("=" * 80)
        print(f"Initial samples: {len(row_meta):,}")
        
        # 检查并解析元数据字段
        print(f"\n📋 Checking metadata fields...")
        print(f"   Available columns: {list(row_meta.columns)}")
        
        # 尝试从不同可能的列名获取必要信息
        if 'pert_id' not in row_meta.columns:
            # 尝试从其他可能的列名获取
            for col in ['perturbation_id', 'pert_iname', 'id']:
                if col in row_meta.columns:
                    print(f"   Using '{col}' as pert_id")
                    row_meta['pert_id'] = row_meta[col]
                    break
            
            # 如果还是找不到，尝试从id列解析
            if 'pert_id' not in row_meta.columns and 'id' in row_meta.columns:
                print(f"   Parsing pert_id from 'id' column...")
                parts = row_meta['id'].str.split('_', expand=True)
                if parts.shape[1] >= 1:
                    row_meta['pert_id'] = parts[0]
        
        if 'cell_id' not in row_meta.columns:
            for col in ['cell_line', 'cell_iname']:
                if col in row_meta.columns:
                    print(f"   Using '{col}' as cell_id")
                    row_meta['cell_id'] = row_meta[col]
                    break
            
            if 'cell_id' not in row_meta.columns and 'id' in row_meta.columns:
                print(f"   Parsing cell_id from 'id' column...")
                parts = row_meta['id'].str.split('_', expand=True)
                if parts.shape[1] >= 2:
                    row_meta['cell_id'] = parts[1]
        
        # 检查必要的列
        required_cols = ['pert_id', 'cell_id']
        missing_cols = [col for col in required_cols if col not in row_meta.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns after parsing: {missing_cols}")
        
        print(f"   ✓ Found required columns: pert_id, cell_id")
        
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()
        initial_compounds = working_meta['pert_id'].nunique()
        
        # ========== Filter 1: Remove DOS compounds ==========
        if remove_dos:
            print(f"\n{'='*80}")
            print(f"FILTER 1: Remove DOS (Diversity-Oriented Synthesis) compounds")
            print(f"{'='*80}")
            
            # 更宽松的DOS识别
            if 'pert_type' in working_meta.columns:
                dos_mask = ~working_meta['pert_type'].str.contains('DOS', case=False, na=False)
            else:
                # 只移除明确标记为DOS的化合物
                dos_mask = ~(
                    working_meta['pert_id'].str.contains('BRD-DOS', case=False, na=False) |
                    working_meta['pert_id'].str.contains('BDOS', case=False, na=False)
                )
            
            n_dos = (~dos_mask).sum()
            working_matrix = working_matrix[dos_mask]
            working_meta = working_meta[dos_mask].reset_index(drop=True)
            
            print(f"  Removed {n_dos:,} DOS observations")
            print(f"  Remaining samples: {len(working_meta):,}")
            print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 2: Minimum observations per compound ==========
        print(f"\n{'='*80}")
        print(f"FILTER 2: Remove compounds with <{min_observations_per_compound} observations")
        print(f"         (relaxed from paper's 5)")
        print(f"{'='*80}")
        
        obs_counts = working_meta.groupby('pert_id').size()
        valid_perts = obs_counts[obs_counts >= min_observations_per_compound].index
        
        print(f"  Compounds with ≥{min_observations_per_compound} observations: "
              f"{len(valid_perts):,}/{obs_counts.nunique():,}")
        
        obs_mask = working_meta['pert_id'].isin(valid_perts)
        working_matrix = working_matrix[obs_mask]
        working_meta = working_meta[obs_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 3: Cosine similarity to closest replicate ==========
        print(f"\n{'='*80}")
        print(f"FILTER 3: Remove observations with cosine similarity <{min_replicate_similarity}")
        print(f"         (relaxed from paper's 0.12)")
        print(f"{'='*80}")
        
        nearest_similarities = self.calculate_cosine_similarity_to_nearest_replicate(
            working_matrix, 
            working_meta['pert_id']
        )
        
        sim_mask = nearest_similarities >= min_replicate_similarity
        n_removed_sim = (~sim_mask).sum()
        
        working_matrix = working_matrix[sim_mask]
        working_meta = working_meta[sim_mask].reset_index(drop=True)
        
        print(f"  Removed {n_removed_sim:,} low-similarity observations")
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Filter 4: Dose selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 4: Select most frequent dose in range {dose_range[0]}-{dose_range[1]} µM")
        print(f"         (relaxed from paper's 1-20 µM)")
        print(f"{'='*80}")
        
        has_dose_info = False
        if 'pert_dose' in working_meta.columns:
            has_dose_info = True
        elif 'pert_idose' in working_meta.columns:
            working_meta['pert_dose'] = working_meta['pert_idose']
            has_dose_info = True
        elif 'id' in working_meta.columns:
            # 尝试从id解析
            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] >= 4:
                working_meta['pert_dose'] = parts[3]
                has_dose_info = True
        
        if has_dose_info:
            # 解析剂量值
            working_meta['dose_value'] = pd.to_numeric(
                working_meta['pert_dose'].astype(str).str.extract(r'(\d+\.?\d*)')[0], 
                errors='coerce'
            )
            
            # 解析剂量单位
            working_meta['dose_unit'] = working_meta['pert_dose'].astype(str).str.extract(r'([a-zA-Z]+)')[0]
            
            # 转换为µM
            def convert_to_uM(row):
                if pd.isna(row['dose_value']):
                    return np.nan
                value = row['dose_value']
                unit = str(row['dose_unit']).lower() if pd.notna(row['dose_unit']) else 'um'
                
                if 'nm' in unit:
                    return value / 1000
                elif 'mm' in unit:
                    return value * 1000
                else:
                    return value
            
            working_meta['dose_uM'] = working_meta.apply(convert_to_uM, axis=1)
            
            # 筛选有效剂量范围
            dose_mask = (
                (working_meta['dose_uM'] >= dose_range[0]) & 
                (working_meta['dose_uM'] <= dose_range[1])
            )
            
            # 对每个化合物，选择最常见的剂量
            valid_doses = []
            for pert_id in working_meta['pert_id'].unique():
                pert_mask = (working_meta['pert_id'] == pert_id) & dose_mask
                if pert_mask.sum() == 0:
                    # 如果没有在范围内的剂量，保留所有剂量
                    if skip_missing_metadata:
                        pert_all_mask = working_meta['pert_id'] == pert_id
                        valid_doses.append(pert_all_mask)
                    continue
                
                # 找出最常见的剂量
                dose_counts = working_meta.loc[pert_mask, 'dose_uM'].value_counts()
                if len(dose_counts) > 0:
                    most_common_dose = dose_counts.index[0]
                    valid_doses.append(
                        (working_meta['pert_id'] == pert_id) & 
                        (working_meta['dose_uM'] == most_common_dose)
                    )
            
            if valid_doses:
                dose_final_mask = pd.concat([pd.Series(mask) for mask in valid_doses], axis=1).any(axis=1)
                n_removed_dose = (~dose_final_mask).sum()
                
                working_matrix = working_matrix[dose_final_mask]
                working_meta = working_meta[dose_final_mask].reset_index(drop=True)
                
                print(f"  Removed {n_removed_dose:,} observations (invalid or non-modal dose)")
                print(f"  Remaining samples: {len(working_meta):,}")
                print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
            else:
                print(f"  Warning: No valid doses found, keeping all data")
        else:
            print(f"  Warning: 'pert_dose' column not found, skipping dose filter")
        
        # ========== Filter 5: Timepoint selection ==========
        print(f"\n{'='*80}")
        print(f"FILTER 5: Keep only measurements at {valid_timepoints}")
        print(f"{'='*80}")
        
        has_time_info = False
        if 'pert_time' in working_meta.columns:
            has_time_info = True
        elif 'pert_itime' in working_meta.columns:
            working_meta['pert_time'] = working_meta['pert_itime']
            has_time_info = True
        elif 'id' in working_meta.columns:
            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] >= 3:
                working_meta['pert_time'] = parts[2]
                has_time_info = True
        
        if has_time_info:
            # 标准化时间点格式（更宽松）
            working_meta['time_normalized'] = working_meta['pert_time'].astype(str).str.lower().str.strip()
            
            # 扩展时间点匹配模式
            valid_patterns = []
            for tp in valid_timepoints:
                valid_patterns.append(tp.lower().strip())
                valid_patterns.append(tp.lower().strip().replace('h', ' h'))
                valid_patterns.append(tp.lower().strip().replace('h', 'hr'))
                valid_patterns.append(tp.lower().strip().replace(' ', ''))
            
            valid_patterns = list(set(valid_patterns))
            time_mask = working_meta['time_normalized'].isin(valid_patterns)
            
            n_removed_time = (~time_mask).sum()
            
            if skip_missing_metadata and n_removed_time == len(working_meta):
                print(f"  Warning: No matching timepoints found, keeping all data")
            else:
                working_matrix = working_matrix[time_mask]
                working_meta = working_meta[time_mask].reset_index(drop=True)
                
                print(f"  Removed {n_removed_time:,} observations (invalid timepoint)")
                print(f"  Remaining samples: {len(working_meta):,}")
                print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        else:
            print(f"  Warning: 'pert_time' column not found, skipping timepoint filter")
        
        # ========== Filter 6: Cell line count per compound ==========
        print(f"\n{'='*80}")
        print(f"FILTER 6: Remove compounds in <{min_cell_lines} or >{max_cell_lines} cell lines")
        print(f"         (relaxed from paper's 5-40)")
        print(f"{'='*80}")
        
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        valid_perts_cell = cell_line_counts[
            (cell_line_counts >= min_cell_lines) & 
            (cell_line_counts <= max_cell_lines)
        ].index
        
        print(f"  Compounds in {min_cell_lines}-{max_cell_lines} cell lines: "
              f"{len(valid_perts_cell):,}/{len(cell_line_counts):,}")
        
        cell_mask = working_meta['pert_id'].isin(valid_perts_cell)
        working_matrix = working_matrix[cell_mask]
        working_meta = working_meta[cell_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta):,}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique():,}")
        
        # ========== Create final dataset ==========
        print(f"\n{'='*80}")
        print(f"✅ FINAL DATASET (After all filters)")
        print(f"{'='*80}")
        
        # 创建化合物标签编码
        unique_perts = sorted(working_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in working_meta['pert_id']])
        
        final_compounds = len(unique_perts)
        final_samples = len(working_matrix)
        final_cells = working_meta['cell_id'].nunique()
        
        print(f"  Total samples: {final_samples:,}")
        print(f"  Total compounds: {final_compounds:,}")
        print(f"  Cell lines: {final_cells}")
        print(f"  Gene features: {working_matrix.shape[1]}")
        print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
        print(f"  Samples per compound (median): "
              f"{working_meta.groupby('pert_id').size().median():.0f}")
        
        # 统计化合物分布
        compound_obs = working_meta.groupby('pert_id').size()
        n_compounds_100plus = (compound_obs > 100).sum()
        print(f"  Compounds with >100 observations: {n_compounds_100plus:,}")
        
        # 与原文对比
        print(f"\n📊 Comparison with paper results:")
        print(f"  Paper: 425,242 obs, 9,597 compounds, 52 cell lines")
        print(f"  Ours:  {final_samples:,} obs, {final_compounds:,} compounds, {final_cells} cell lines")
        
        if initial_compounds > 0:
            retention_rate = (final_compounds / initial_compounds) * 100
            print(f"  Compound retention rate: {retention_rate:.1f}%")
        
        training_data = {
            'X': working_matrix,
            'y': labels,
            'sample_meta': working_meta,
            'gene_names': col_meta['id'].values,
            'compound_names': unique_perts,
            'pert_to_idx': pert_to_idx
        }
        
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*80}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*80}")
        
        folds = np.zeros(len(sample_meta), dtype=int)
        
        # 按化合物分配fold
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


def main():
    """主程序 - 放宽筛选条件版本"""
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    try:
        # Step 1: 加载基因和细胞信息
        print("=" * 80)
        print("STEP 1: Loading gene and cell information")
        print("=" * 80)
        gene_info = loader.load_gene_info('GSE92742')
        cell_info = loader.load_cell_info('GSE92742')
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 80)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 80)
        matrix, row_meta, col_meta = loader.load_level4_signatures('GSE92742')
        
        # Step 3: 准备训练数据（放宽条件）
        print("\n" + "=" * 80)
        print("STEP 3: Preparing training data (Relaxed filters)")
        print("=" * 80)
        training_data = loader.prepare_training_data(
            min_observations_per_compound=3,    # 放宽：5 → 3
            min_replicate_similarity=0.10,      # 放宽：0.12 → 0.10
            dose_range=(0.3, 30.0),             # 放宽：1-20 → 0.3-30
            valid_timepoints=['6h', '24h', '6 h', '24 h'],
            min_cell_lines=3,                   # 放宽：5 → 3
            max_cell_lines=50,                  # 放宽：40 → 50
            remove_dos=True,
            skip_missing_metadata=True
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 80)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 80)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存处理后的数据
        print("\n" + "=" * 80)
        print("STEP 5: Saving processed data")
        print("=" * 80)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True)
        
        output_file = output_dir / "training_data_relaxed.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        
        # 打印最终摘要
        print("\n" + "=" * 80)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 80)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Data shape: {training_data['X'].shape}")
        print(f"   • Average samples per compound: "
              f"{len(training_data['X']) / len(training_data['compound_names']):.1f}")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 80)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 80)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        print(f"\n📋 Full traceback:")
        import traceback
        traceback.print_exc()
        return None


if __name__ == "__main__":
    training_data = main()

STEP 1: Loading gene and cell information
Total genes: 12328
Landmark genes: 978
Total cell lines: 98
Unique cell IDs: 98

STEP 2: Loading Level 4 signatures
🔍 Searching for Level 4 file...
   Pattern: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n*x12328.gctx.gz
✓ Found: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx.gz

📖 Reading GCTX file: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx.gz
⚠️  Detected gzip compressed file
✓ Found existing decompressed file: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx
📊 Loading matrix from HDF5...
✓ Matrix shape: (1319138, 12328)
📋 Loading metadata...

✓ Data loaded successfully:
  Matrix: (1319138, 12328) (samples × genes)
  Samples: 1319138
  Genes: 12328

🔬 Filtering to landmark genes...
   Landmark genes to find: 978
   Total genes in data: 12328
   ✓ Matched: 978 genes

🎯 Applying filter...
   ✓ Final matrix shape: (1319138, 978)

STEP 3: Preparing training data (Relaxed filters)

🔍 DRUGREFLECTOR 

Traceback (most recent call last):
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_28812\743581544.py", line 656, in main
    training_data = loader.prepare_training_data(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_28812\743581544.py", line 563, in prepare_training_data
    print(f"  Samples per compound (mean): {final_samples / final_compounds:.1f}")
                                            ~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~
ZeroDivisionError: division by zero


: 